# AML Multi-GNN - Clean Training Script

A robust, bug-free training script for Multi-GNN AML detection.
**No synthetic data - only real data training with proper error handling.**


In [ ]:
#!/usr/bin/env python3
"""
AML Multi-GNN - Clean Training Script
=====================================

A robust, bug-free training script for Multi-GNN AML detection.
No synthetic data - only real data training with proper error handling.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import os
import gc
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")


In [ ]:
# Set device with proper error handling
def get_device():
    """Get the best available device with proper error handling"""
    if torch.cuda.is_available():
        try:
            # Test CUDA with a simple operation
            test_tensor = torch.tensor([1.0]).cuda()
            _ = test_tensor + 1
            torch.cuda.empty_cache()
            return torch.device('cuda')
        except Exception as e:
            print(f"CUDA test failed: {e}")
            print("Falling back to CPU")
            return torch.device('cpu')
    else:
        return torch.device('cpu')

device = get_device()
print(f"Using device: {device}")

# Clear GPU memory
if device.type == 'cuda':
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print("GPU memory cleared")


In [ ]:
class SimpleGNN(nn.Module):
    """Simple, robust GNN for AML detection"""
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, dropout=0.1):
        super(SimpleGNN, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        # Input layer
        self.input_conv = GCNConv(input_dim, hidden_dim)
        
        # Hidden layers
        self.hidden_convs = nn.ModuleList()
        for _ in range(num_layers - 1):
            self.hidden_convs.append(GCNConv(hidden_dim, hidden_dim))
        
        # Output layer
        self.output_conv = GCNConv(hidden_dim, output_dim)
        
        # Dropout
        self.dropout_layer = nn.Dropout(dropout)
        
    def forward(self, x, edge_index, batch=None):
        """Forward pass with proper device handling"""
        # Ensure all inputs are on the same device
        x = x.to(device)
        edge_index = edge_index.to(device)
        
        # Input layer
        x = F.relu(self.input_conv(x, edge_index))
        x = self.dropout_layer(x)
        
        # Hidden layers
        for conv in self.hidden_convs:
            x = F.relu(conv(x, edge_index))
            x = self.dropout_layer(x)
        
        # Output layer
        x = self.output_conv(x, edge_index)
        
        # CRITICAL FIX: Always use global pooling for graph-level classification
        if batch is not None:
            batch = batch.to(device)
            x = global_mean_pool(x, batch)
        else:
            # If no batch provided, assume single graph and use mean pooling
            x = x.mean(dim=0, keepdim=True)
        
        return x

print("SimpleGNN model class defined")


In [ ]:
def load_real_data(data_path):
    """Load real AML data with proper error handling - NO SYNTHETIC DATA"""
    print("Loading real AML data...")
    
    # Check for real data files
    real_files = []
    for file in os.listdir(data_path):
        if file.endswith('.csv'):
            real_files.append(file)
    
    print(f"Found {len(real_files)} CSV files in {data_path}")
    for file in real_files:
        print(f"  - {file}")
    
    if not real_files:
        raise ValueError("NO REAL DATA FOUND! Please ensure your data is in Google Drive at /content/drive/MyDrive/LaunDetection/data/raw/")
    
    # Use HI-Small dataset for better performance
    hi_small_trans = None
    hi_small_accounts = None
    
    for file in real_files:
        if 'HI-Small_Trans' in file:
            hi_small_trans = file
        elif 'HI-Small_accounts' in file:
            hi_small_accounts = file
    
    if hi_small_trans:
        transactions_file = os.path.join(data_path, hi_small_trans)
        print(f"Loading transactions from {transactions_file}")
        transactions = pd.read_csv(transactions_file, nrows=2000)  # Limit for memory
        print(f"Loaded {len(transactions)} transactions")
    else:
        # Fallback to first file
        transactions_file = os.path.join(data_path, real_files[0])
        print(f"Loading transactions from {transactions_file}")
        transactions = pd.read_csv(transactions_file, nrows=2000)  # Limit for memory
        print(f"Loaded {len(transactions)} transactions")
    
    if hi_small_accounts:
        accounts_file = os.path.join(data_path, hi_small_accounts)
        print(f"Loading accounts from {accounts_file}")
        accounts = pd.read_csv(accounts_file, nrows=1000)  # Limit for memory
        print(f"Loaded {len(accounts)} accounts")
    else:
        # Use only real data - extract accounts from transaction data
        print("Extracting real accounts from transaction data...")
        all_accounts = set(transactions['From Bank'].tolist() + transactions['To Bank'].tolist())
        
        # Create accounts using real bank names from transactions
        accounts_data = {
            'Account Number': list(all_accounts),
            'Bank Name': list(all_accounts),  # Use real bank names
            'Bank ID': [f'B{hash(name) % 10000}' for name in all_accounts],  # Generate IDs from bank names
            'Entity ID': [f'E{hash(name) % 10000}' for name in all_accounts],
            'Entity Name': [f'Entity_{name}' for name in all_accounts]
        }
        accounts = pd.DataFrame(accounts_data)
        print(f"Extracted {len(accounts)} real accounts from transaction data")
    
    return transactions, accounts

# Removed synthetic data creation functions - using only real data

print("Data loading functions defined")


In [ ]:
def create_graph_from_data(transactions, accounts):
    """Create graph from transaction and account data - REAL DATA ONLY"""
    print("Creating graph from real data...")
    
    # Handle different column names in real data
    print("Transaction columns:", transactions.columns.tolist())
    print("Account columns:", accounts.columns.tolist())
    
    # Map common column names
    from_col = None
    to_col = None
    amount_col = None
    timestamp_col = None
    sar_col = None
    
    # Find transaction columns
    for col in transactions.columns:
        col_lower = col.lower()
        if 'from' in col_lower or 'sender' in col_lower:
            from_col = col
        elif 'to' in col_lower or 'receiver' in col_lower:
            to_col = col
        elif 'amount' in col_lower or 'value' in col_lower:
            amount_col = col
        elif 'time' in col_lower or 'date' in col_lower:
            timestamp_col = col
        elif 'laundering' in col_lower or 'sar' in col_lower or 'suspicious' in col_lower or 'label' in col_lower:
            sar_col = col
    
    print(f"Using columns: from={from_col}, to={to_col}, amount={amount_col}, time={timestamp_col}, sar={sar_col}")
    
    # Create node features - handle real IBM AML data structure
    account_features = {}
    
    # Map account columns to our expected format
    account_id_col = None
    balance_col = None
    risk_col = None
    type_col = None
    
    # Find account ID column
    for col in accounts.columns:
        col_lower = col.lower()
        if 'account' in col_lower and 'number' in col_lower:
            account_id_col = col
        elif 'balance' in col_lower or 'amount' in col_lower:
            balance_col = col
        elif 'risk' in col_lower or 'score' in col_lower:
            risk_col = col
        elif 'type' in col_lower or 'entity' in col_lower:
            type_col = col
    
    print(f"Account column mapping: id={account_id_col}, balance={balance_col}, risk={risk_col}, type={type_col}")
    
    for _, account in accounts.iterrows():
        # Use Account Number as account_id
        account_id = account[account_id_col] if account_id_col else account.iloc[0]
        
        # Extract features with defaults
        balance = float(account[balance_col]) if balance_col and pd.notna(account[balance_col]) else 5000.0
        risk_score = float(account[risk_col]) if risk_col and pd.notna(account[risk_col]) else 0.5
        account_type = str(account[type_col]) if type_col and pd.notna(account[type_col]) else 'checking'
        
        account_features[account_id] = [
            balance,
            risk_score,
            1 if account_type.lower() == 'checking' else 0,
            1 if account_type.lower() == 'savings' else 0,
            1 if account_type.lower() == 'business' else 0
        ]
    
    # Create edge features and labels
    edges = []
    edge_features = []
    labels = []
    
    print(f"Processing {len(transactions)} transactions...")
    print(f"Account features keys: {list(account_features.keys())[:5]}...")  # Show first 5 account IDs
    
    matched_edges = 0
    for i, (_, transaction) in enumerate(transactions.iterrows()):
        if i % 1000 == 0:
            print(f"Processing transaction {i}/{len(transactions)}")
            
        # Handle the specific IBM AML data structure
        from_acc = transaction[from_col] if from_col else transaction.iloc[0]
        to_acc = transaction[to_col] if to_col else transaction.iloc[1]
        
        # Use Amount Paid as the transaction amount
        amount = float(transaction[amount_col]) if amount_col and pd.notna(transaction[amount_col]) else 1000.0
        
        # Handle timestamp
        if timestamp_col and timestamp_col in transaction:
            try:
                if pd.api.types.is_datetime64_any_dtype(transaction[timestamp_col]):
                    timestamp = transaction[timestamp_col]
                else:
                    timestamp = pd.to_datetime(transaction[timestamp_col])
                hour = timestamp.hour
                day = timestamp.day
                month = timestamp.month
            except:
                hour, day, month = 12, 1, 1
        else:
            hour, day, month = 12, 1, 1
        
        # Handle SAR label - use ONLY real "Is Laundering" column
        if sar_col and sar_col in transaction:
            is_sar = int(transaction[sar_col])
        else:
            # No synthetic labels - skip if no real SAR data
            print(f"WARNING: No SAR label found for transaction {i}, skipping...")
            continue
        
        # Debug: Check if accounts exist
        if i < 5:  # Debug first 5 transactions
            print(f"Transaction {i}: from_acc={from_acc}, to_acc={to_acc}")
            print(f"  from_acc in account_features: {from_acc in account_features}")
            print(f"  to_acc in account_features: {to_acc in account_features}")
        
        # ONLY use real data - skip transactions with missing accounts
        if from_acc in account_features and to_acc in account_features:
            edges.append([from_acc, to_acc])
            edge_features.append([amount, hour, day, month])
            labels.append(is_sar)
            matched_edges += 1
        else:
            # Skip transactions with missing accounts - use only real data
            if i < 5:  # Debug first 5 skipped transactions
                print(f"  ✗ Skipped transaction {i}: missing accounts")
                print(f"    from_acc '{from_acc}' in account_features: {from_acc in account_features}")
                print(f"    to_acc '{to_acc}' in account_features: {to_acc in account_features}")
            continue
    
    print(f"Matched {matched_edges} edges out of {len(transactions)} transactions")
    
    # Create node feature matrix
    unique_accounts = list(set([edge[0] for edge in edges] + [edge[1] for edge in edges]))
    node_features = []
    node_labels = []
    
    for account_id in unique_accounts:
        if account_id in account_features:
            node_features.append(account_features[account_id])
            # Node label: 1 if any transaction from this account is SAR
            account_sar = any(labels[i] for i, edge in enumerate(edges) if edge[0] == account_id)
            node_labels.append(1 if account_sar else 0)
        else:
            node_features.append([0, 0, 0, 0, 0])
            node_labels.append(0)
    
    # Convert to tensors
    node_features = torch.tensor(node_features, dtype=torch.float32)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_features, dtype=torch.float32)
    labels = torch.tensor(node_labels, dtype=torch.long)
    
    print(f"Created graph with {len(unique_accounts)} nodes and {len(edges)} edges")
    
    # Handle division by zero
    if len(node_labels) > 0:
        sar_rate = sum(node_labels) / len(node_labels)
        print(f"SAR rate: {sar_rate:.3f}")
    else:
        print("SAR rate: 0.000 (no nodes)")
    
    # Use only real data - no synthetic fallback
    if len(edges) == 0:
        print("ERROR: No edges created from real data!")
        print("This means there's no overlap between transaction accounts and account data.")
        print("Please check your data files and ensure account IDs match.")
        raise ValueError("No real data available for training - check data files")
    else:
        # Convert to tensors using only real data
        node_features = torch.tensor(node_features, dtype=torch.float32)
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_features, dtype=torch.float32)
        labels = torch.tensor(node_labels, dtype=torch.long)
    
    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=labels)

print("Graph creation function defined")


In [ ]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001):
    """Train model with proper device handling"""
    print(f"Training model for {epochs} epochs...")
    
    # Move model to device
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    best_val_f1 = 0.0
    train_losses = []
    val_f1_scores = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        total_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            # Move batch to device
            batch = batch.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = criterion(out, batch.y)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                
                val_preds.extend(pred.cpu().numpy())
                val_labels.extend(batch.y.cpu().numpy())
        
        # Calculate metrics
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        val_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_recall = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
        
        train_losses.append(total_loss / len(train_loader))
        val_f1_scores.append(val_f1)
        
        print(f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, "
              f"Val F1={val_f1:.4f}, Val Precision={val_precision:.4f}, Val Recall={val_recall:.4f}")
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            print(f"New best validation F1: {best_val_f1:.4f}")
    
    return best_val_f1, train_losses, val_f1_scores

print("Training function defined")


In [ ]:
# Main training execution
print("="*60)
print("AML Multi-GNN - Clean Training Script")
print("="*60)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load data
data_path = "/content/drive/MyDrive/LaunDetection/data/raw"
transactions, accounts = load_real_data(data_path)

# Create graph
graph = create_graph_from_data(transactions, accounts)
print(f"Graph created: {graph.num_nodes} nodes, {graph.num_edges} edges")


In [ ]:
# Create data loader
from torch_geometric.loader import DataLoader as PyGDataLoader

# CRITICAL FIX: Create individual graphs for each node (graph-level classification)
print("Creating individual graphs for graph-level classification...")

# Create individual graphs for each node
individual_graphs = []
for i in range(min(100, graph.num_nodes)):  # Limit to 100 graphs for memory
    # Create a small subgraph around each node
    center_node = i
    neighbor_nodes = [center_node]
    
    # Add some neighbors
    for edge_idx in range(graph.edge_index.size(1)):
        edge = graph.edge_index[:, edge_idx]
        if edge[0].item() == center_node:
            neighbor_nodes.append(edge[1].item())
        elif edge[1].item() == center_node:
            neighbor_nodes.append(edge[0].item())
    
    # Limit to reasonable size
    neighbor_nodes = neighbor_nodes[:10]
    
    if len(neighbor_nodes) > 1:  # Only create if has neighbors
        # Create subgraph
        subgraph = graph.subgraph(torch.tensor(neighbor_nodes))
        if subgraph.num_edges > 0:
            # Create graph-level label (majority vote of node labels)
            node_labels = subgraph.y.tolist()
            graph_label = 1 if sum(node_labels) > len(node_labels) / 2 else 0
            
            # Create new graph with single label
            new_graph = Data(
                x=subgraph.x,
                edge_index=subgraph.edge_index,
                edge_attr=subgraph.edge_attr,
                y=torch.tensor([graph_label], dtype=torch.long)
            )
            individual_graphs.append(new_graph)

print(f"Created {len(individual_graphs)} individual graphs")

# Split into train/val
train_size = int(0.8 * len(individual_graphs))
train_graphs = individual_graphs[:train_size]
val_graphs = individual_graphs[train_size:]

print(f"Split: {len(train_graphs)} train graphs, {len(val_graphs)} val graphs")

# Create data loaders
train_loader = PyGDataLoader(train_graphs, batch_size=4, shuffle=True)
val_loader = PyGDataLoader(val_graphs, batch_size=4, shuffle=False)

print(f"Data loaders created: {len(train_loader)} train batches, {len(val_loader)} val batches")


In [ ]:
# Create model
model = SimpleGNN(
    input_dim=5,  # 5 node features
    hidden_dim=32,
    output_dim=2,  # Binary classification
    num_layers=2,
    dropout=0.1
)

print(f"Model created with {sum(p.numel() for p in model.parameters())} parameters")

# Train model
best_f1, train_losses, val_f1_scores = train_model(
    model, train_loader, val_loader, epochs=10, lr=0.001
)

print(f"\nTraining completed!")
print(f"Best validation F1: {best_f1:.4f}")


In [ ]:
# Test model
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        pred = out.argmax(dim=1)
        
        test_preds.extend(pred.cpu().numpy())
        test_labels.extend(batch.y.cpu().numpy())

# Final metrics
test_f1 = f1_score(test_labels, test_preds, average='weighted')
test_precision = precision_score(test_labels, test_preds, average='weighted', zero_division=0)
test_recall = recall_score(test_labels, test_preds, average='weighted', zero_division=0)

print(f"\nFinal Test Results:")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")

print("\n" + "="*60)
print("Training completed successfully!")
print("="*60)
